## Creating custom metrics for your Large Language Model (LLM)

Within this Jupyter notebook, you'll discover the seamless process of crafting and implementing custom metrics using code. This invaluable functionality empowers users to deploy metrics tailored precisely to their unique business requirements. Whether you're tracking specialized metrics or catering to distinct business needs, this tutorial equips you with the tools to seamlessly integrate custom metrics into your workflow.

## Installation
Ensure you have the required libraries installed by executing the following commands in your Jupyter notebook:

In [ ]:
!pip install -q datarobot datarobotx[llm] tiktoken textstat textblob nltk py-readability-metrics

In [ ]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /home/notebooks/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/notebooks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/notebooks/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/notebooks/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     /home/notebooks/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/notebooks/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


## Creating custom metric function
To understand the Total Cost of the solution, the organization will build a metric around the pricing of the GPT 3.5 API provided by Azure. The following function calculates the price per prediction call using token counts calculated using Tiktoken. It then multiplies the token counts with the price per token provided in the Azure OpenAl pricing
page. If the LLM is self-hosted, metrics around the compute cost are relevant for the organization.

In [ ]:
import tiktoken, os


# Obtain encoding for the specified model
encoding = tiktoken.get_encoding("cl100k_base")

# Define custom metrics functions
def get_gpt_token_count(text):
    return len(encoding.encode(text))

def get_gpt_3_5_cost(
    prompt, response, prompt_token_cost=0.0015 / 1000, response_token_cost=0.002 / 1000
):
    return (
        get_gpt_token_count(prompt) * prompt_token_cost
        + get_gpt_token_count(response) * response_token_cost
    )

# Example usage
prompt = "How can we improve renewable energy?"
response = "By investing in solar and wind technologies."

# Calculate custom metric
cost = get_gpt_3_5_cost(prompt, response)
print("Cost of generating response:", cost)


## Upload custom metrics
This function submit_custom_metric records values for an existing custom metric on a deployment. It takes the API URL, API key, deployment ID, custom metric ID, model package ID, and the metric value as inputs, and posts the data to the specified endpoint.

In [ ]:
API_URL = os.environ['API_URL']
API_KEY = os.environ['API_KEY']
DEPLOYMENT_ID = os.environ['DEPLOYMENT_ID']
CUSTOM_METRIC_ID = os.environ['CUSTOM_METRIC_ID']
MODEL_PACKAGE_ID = os.environ['MODEL_PACKAGE_ID']

def submit_custom_metric(API_URL,API_KEY,DEPLOYMENT_ID,CUSTOM_METRIC_ID,MODEL_PACKAGE_ID, metric):
    """Record values for an existing custom metric on a deployment"""
    
    HEADERS = {
    'Authorization': 'Bearer {}'.format(API_KEY),
    'User-Agent': 'IntegrationSnippet-Requests',
    }
    time_ = datetime.today().strftime("%m/%d/%Y %I:%M %p")
    rows = [
        {"timestamp": ts.isoformat(), "value": value}
        for ts, value in zip([pd.to_datetime(time_)], [metric])
    ]
    response = requests.post(
        API_URL.format(DEPLOYMENT_ID, CUSTOM_METRIC_ID),
        json={'modelPackageId': MODEL_PACKAGE_ID, 'buckets': rows,},
        headers=HEADERS,
    )
    response.raise_for_status()

In [ ]:
submit_custom_metric(
    API_URL,
    API_KEY,
    DEPLOYMENT_ID,
    CUSTOM_METRIC_ID,
    MODEL_PACKAGE_ID,
    get_gpt_3_5_cost(inputText, predictions_response.json()["data"][0]["prediction"]),
)